# セグメント木
「○○番目から△△番目までの要素の最大値を求めてください」 のような**区間に関するクエリを得意とするデータ構造**

### 実装する際の特徴
- セグメント木の要素数(大きさ)は、ターゲット N の数を超える最小の $2^x$ の数。  
  N = 5 の時、 セグメント木のサイズは $2^3 = 8$
- 一番上の要素の番号は1(0番目のデータは使わない)
- 要素 u の左下の番号は 2u
- 要素 u の右下の番号は 2u+1
- 求めたい区間が与えられれば、それを含むセグメント木の中の大きい区間から順番に再帰的に探索する

## RMQ(Range Maximum Querires)
セグメント木で区間の最大値を記録しておく問題

### C++ での実装
```cpp
class SegmentTree {
 public:
  int data[300000], size = 1;

  void init(int N) {
    size = 1;
    // N を超える最小の 2 の累乗がサイズとなる
    while (size < N) size *= 2;
    rep(i, 1, size * 2) data[i] = 0;
  }

  void update(int pos, int x) {
    pos = pos + size - 1;
    data[pos] = x;
    while (pos >= 2) {
      pos /= 2;
      data[pos] = max(data[pos * 2], data[pos * 2 + 1]);
    }
  }

  int query(int l, int r, int a, int b, int u) {
    if (r <= a || b <= l) return -1000000000;
    if (l <= a && b <= r) return data[u];  // 完全に含まれる場合
    int m = (a + b) / 2;
    int ans_l = query(l, r, a, m, u * 2);
    int ans_r = query(l, r, m, b, u * 2 + 1);
    return max(ans_l, ans_r);
  }
};
```

### Python での実装

In [1]:
class SegmentTree:
    def __init__(self):
        self.dat = []
        self.size = 1

    def init(self, N):
        self.size = 1
        while self.size < N:
            self.size *= 2
        # Aは最初全ての要素が0
        self.dat = [0]*(self.size*2)

    def update(self, pos, x):
        # self.size//2 以降が、最も下の要素(葉)である
        pos = pos + self.size
        self.dat[pos] = x
        # 上のノードへの影響を反映する
        while pos >= 2:
            # 親ノードのindex
            pos //= 2
            # 二つの子ノードの大きい方を親ノードに反映する
            self.dat[pos] = max(self.dat[pos*2], self.dat[pos*2+1])

    # (l, r)は目標区間、(a, b)は現在調査中の区間、u はセル番号、
    def query(self, l, r, a, b, u):
        # print(l, r, a, b, u, self.dat[u])
        # 全く含まれていない場合
        if b <= l or r <= a:
            return -math.inf
        # 完全に含間れている場合
        if l <= a and b <= r:
            return self.dat[u]
        mid = (a+b)//2
        left_ans = self.query(l, r, a, mid, u*2)
        right_ans = self.query(l, r, mid, b, u*2+1)
        return max(left_ans, right_ans)


def main():
    N, Q = map(int, stdin.readline().split())

    seg_tree = SegmentTree()
    seg_tree.init(N)

    for _ in range(Q):
        *query, = map(int, stdin.readline().split())
        if query[0] == 1:
            pos, x = query[1], query[2]
            seg_tree.update(pos-1, x)
        elif query[0] == 2:
            l, r = query[1], query[2]
            ans = seg_tree.query(l-1, r-1, 0, seg_tree.size, 1)
            print(ans)

## RSQ(Range Sum Querires)
セグメント木で区間の和を記録しておく問題

### C++ での実装
```cpp
class SegmentTree {
 public:
  int data[300009], size = 1;
  void init(int N) {
    size = 1;
    while (size < N) {
      size *= 2;
    }
    rep(i, 1, size * 2 + 1) data[i] = 0;
  }

  void update(int pos, int x) {
    pos = pos + size - 1;
    data[pos] = x;
    while (pos > 1) {
      pos /= 2;
      data[pos] = data[pos * 2] + data[pos * 2 + 1];
    }
  }

  int query(int l, int r, int a, int b, int pos) {
    if (l <= a && b <= r) return data[pos];
    if (b <= l || r <= a) return 0;
    int mid = (a + b) / 2;
    int l_sum = query(l, r, a, mid, pos * 2);
    int r_sum = query(l, r, mid, b, pos * 2 + 1);
    return l_sum + r_sum;
  }
};
```
### Python での実装

In [2]:
class SegmentTree:
    def __init__(self):
        self.dat = []
        self.size = 1

    def init(self, N):
        self.size = 1
        while self.size < N:
            self.size *= 2
        # Aは最初全ての要素が0
        self.dat = [0]*(self.size*2)

    def update(self, pos, x):
        # self.size//2 以降が、最も下の要素(葉)である
        pos = pos + self.size
        self.dat[pos] = x
        # 上のノードへの影響を反映する
        while pos >= 2:
            # 親ノードのindex
            pos //= 2
            # 二つの子ノードの大きい方を親ノードに反映する
            self.dat[pos] = self.dat[pos*2] + self.dat[pos*2+1]

    # (l, r)は目標区間、(a, b)は現在調査中の区間、u はセル番号、
    def query(self, l, r, a, b, u):
        # print(l, r, a, b, u, self.dat[u])
        # 全く含まれていない場合
        if b <= l or r <= a:
            return 0
        # 完全に含間れている場合
        if l <= a and b <= r:
            return self.dat[u]
        mid = (a+b)//2
        left_ans = self.query(l, r, a, mid, u*2)
        right_ans = self.query(l, r, mid, b, u*2+1)
        return left_ans+right_ans


def main():
    N, Q = map(int, stdin.readline().split())

    seg_tree = SegmentTree()
    seg_tree.init(N)

    for _ in range(Q):
        *query, = map(int, stdin.readline().split())
        if query[0] == 1:
            pos, x = query[1], query[2]
            seg_tree.update(pos-1, x)
        elif query[0] == 2:
            l, r = query[1], query[2]
            ans = seg_tree.query(l-1, r-1, 0, seg_tree.size, 1)
            print(ans)